<a href="https://colab.research.google.com/github/datamino/PINNs/blob/main/case1_p1_(LHOF).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

In [ ]:


# Define the Neural Network
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.fc1 = nn.Linear(1, 20)  # Input layer (x -> hidden)
        self.fc2 = nn.Linear(20, 20)  # Hidden layer
        self.fc3 = nn.Linear(20, 1)   # Output layer (hidden -> y)
        self.activation = torch.tanh  # Activation function

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.fc3(x)
        return x

# Compute the loss function
def compute_loss(model, x):
    x.requires_grad = True  # Enable automatic differentiation
    y_pred = model(x)  # Neural network output
    dy_dx = torch.autograd.grad(y_pred, x, torch.ones_like(y_pred), create_graph=True)[0]
    equation_loss = torch.mean((dy_dx - 2*x) ** 2)  # Enforcing dy/dx = 2x

    # Initial condition loss: y(0) = 1
    x0 = torch.tensor([[0.0]])  # Initial condition point
    y0_pred = model(x0)
    initial_loss = (y0_pred - 1) ** 2

    total_loss = equation_loss + initial_loss
    return total_loss

# Training the model
model = PINN()
optimizer = optim.Adam(model.parameters(), lr=0.01)

x_train = torch.linspace(-2, 2, 100).view(-1, 1)  # Training points

loss_history = []
for epoch in range(1000):
    optimizer.zero_grad()
    loss = compute_loss(model, x_train)
    loss.backward()
    optimizer.step()
    loss_history.append(loss.item())
    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item():.6f}')

# Plot results
x_test = torch.linspace(-2, 2, 100).view(-1, 1)
y_pred = model(x_test).detach().numpy()
y_true = x_test.numpy()**2 + 1  # True solution y = x^2 + 1

plt.plot(x_test.numpy(), y_true, label='True Solution', linewidth=2)
plt.plot(x_test.numpy(), y_pred, label='PINN Approximation', linestyle='dashed')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.title('Solving dy/dx = 2x using PINNs')
plt.show()